In [32]:
# 📦 Import required libraries
import requests  # To send HTTP requests
from lxml import etree  # To parse HTML content using XPath
import pandas as pd  # To organize and save scraped data into Excel

In [34]:
# 🛠️ Connect to a web page and download its content
def conn_web(url):
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)  # Send a GET request with headers
        response.raise_for_status()  # Raise an error if status code is not 200
        content = response.content.decode('UTF-8')  # Decode the content from bytes to UTF-8 string
        return content
    except requests.exceptions.RequestException as e:
        print(f"Error connecting to {url}: {e}")
        return None  # Return None if there’s any connection error


In [36]:
# 🛠️ Extract post information and the next page URL using XPath
def xpath_anchoring(content, data_list):
    try:
        html = etree.HTML(content)  # Parse the HTML content into a tree structure
    except Exception as e:
        print(f"Error parsing HTML content: {e}")
        return data_list, None  # Return safely if parsing fails

    # Locate big post elements (best practice: find the parent first)
    posts = html.xpath("//div[@class='r-list-container action-bar-margin bbs-screen']/div[@class='r-ent']")

    # Try to locate the next page link
    lastPage_url = html.xpath("//div[@class = 'btn-group btn-group-paging']/a[2]/@href")
    lastPage_url = f"https://www.ptt.cc{lastPage_url[0]}" if lastPage_url else None

    # Loop through each post block and extract small fields
    for post in posts:
        title = post.xpath("./div[@class='title']/a/text()")
        title = title[0] if title else None  # Handle missing title

        link = post.xpath("./div[@class='title']/a/@href")
        page_link = f"https://www.ptt.cc{link[0]}" if link else None  # Handle missing link

        author = post.xpath("./div[@class='meta']/div[@class='author']/text()")
        author = author[0] if author else None  # Handle missing author

        date = post.xpath("./div[@class='meta']/div[@class='date']/text()")
        date = date[0] if date else None  # Handle missing date

        # Save extracted post data into the data list
        post_data = {
            "title": title,
            "page_link": page_link,
            "author": author,
            "date": date
        }
        data_list.append(post_data)

    return data_list, lastPage_url  # Return the updated data list and the next page URL

In [38]:
# 🛠️ Scrape multiple pages
def public_static_pages_scraping(next_url, page_num):
    data_list = []

    for i in range(page_num):
        content = conn_web(next_url)  # Connect to the current page
        if content is None:
            print(f"Skipping page {i} due to connection error.")  # Skip if connection fails
            continue
        data_list, next_url = xpath_anchoring(content, data_list)  # Extract posts and get the next page URL
        
        if not next_url:
            break  # Stop if no next page is found

    return data_list

In [40]:
# 🚀 Start scraping
url = "https://www.ptt.cc/bbs/Stock/index.html"  # Starting URL (current stock board page)
pageNum = 10  # How many pages you want to scrape

data_list = public_static_pages_scraping(url, pageNum)  # Scrape the data
data_list = pd.DataFrame(data_list)  # Convert the list of dicts into a DataFrame

# 🛠️ Transform the page_link into Excel clickable hyperlinks
data_list["page_link"] = data_list.apply(
    lambda x: f'=HYPERLINK("{x["page_link"]}", "{x["page_link"]}")' if pd.notna(x["page_link"]) else "",
    axis=1
)

# 🛠️ Save the DataFrame into an Excel file
try:
    data_list.to_excel("output.xlsx", index=False)
    print("Excel file saved successfully!")
except Exception as e:
    print(f"Error saving Excel file: {e}")  # Handle errors during Excel writing


Excel file saved successfully!
